# Test pour prédire le type de cancer (microarray data)
Flückiger Nathan, Vincent Guidoux et Joel Kaufmann

## Préparation des données

### Importation des bibliothèques

In [1]:
import os
import numpy as np
import pylab as plt
import pandas as pd

from Bio import Geo


%matplotlib inline

### Importation des données

In [2]:
def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

In [3]:
records = load_geo('GDS4296.soft')

# description of each condition (cell type)
id_cond = {}  

cancer_set = set()

# loop over records
for r in records:    
    
    rea = r.entity_attributes
    
    #if rea.has_key('subset_sample_id'): 
    if 'subset_sample_id' in rea:
        
        # find conditions associated with each cell type
        if rea['subset_type']=='disease state':
            sample_list = rea['subset_sample_id'].split(',')
            desc = rea['subset_description']
            cancer_set.add(desc)
            for sample in sample_list:
                id_cond[sample] = desc

    #if rea.has_key('dataset_table_begin'): # it's the gene expression data
    if 'dataset_table_begin' in rea:
        data = r.table_rows

# converts data to an array (more practical to handle)
data = np.array(data) 

**Combien y'a-t-il de cellules cancereuses en tout ?**

In [4]:
len(id_cond)

174

**Comment sont-elles réparties ?**

In [5]:
disease_repartition = {}

for disease in id_cond.values():
    if disease in disease_repartition.keys():
        disease_repartition[disease] = disease_repartition[disease] + 1
    else:
        disease_repartition[disease] = 1
        
disease_repartition

{'Adenocarcinoma': 60,
 'Acute lymphoblastic leukemia': 6,
 'Astrocytoma': 3,
 'Chronic myelogenous leukemia': 3,
 'Carcinoma': 12,
 'Carcinosarcoma': 3,
 'Clear cell carcinoma': 2,
 'Cystoadenocarcinoma': 3,
 'Ductal carcinoma': 3,
 'Glial cell neoplasm': 3,
 'Glioblastoma': 12,
 'Large cell carcinoma': 6,
 'Lymphoma': 3,
 'Malignant amelanotic melanoma': 3,
 'Malignant melanotic melanoma': 11,
 'Melanotic melanoma': 9,
 'Myeloma': 3,
 'Papillary infiltrating ductal carcinoma': 3,
 'Pro myelocytic leukemia': 3,
 'Renal spindle cell carcinoma': 3,
 'Renal cell carcinoma': 9,
 'Small cell bronchioalveolar carcinoma': 3,
 'Squamous cell carcinoma': 2,
 'Hypernephroma': 3,
 'Infiltrating ductal carcinoma': 3}

Aux vues de cette répartition, il nous faut regrouper ces différentes type de cellules, car certains, comme `Squamous cell carcinoma` n'ont que deux cellules, et on ne peut pas se permettre d'en mettre une dans le train-set et une dans le test-set, ce n'est pas assez. Nous allons nous renseigner comment regrouper ces données.

In [6]:
def category_repartition(category):
    """Vous nous aider à voir la répartition des différents échantillons dans les catégories que nous allons définir"""
    total = 0
    for disease in category:
        total += disease_repartition[disease]
        
    return total

**5 catégories ont été choisies :**

Adenocarcinoma :

In [7]:
category_1 = ['Adenocarcinoma']

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_1)))

Il y a 60 échantillons dans cette catégorie


Other Carcinoma :

In [8]:
category_2 = [
                'Carcinoma',
                'Carcinosarcoma',
                'Clear cell carcinoma',
                'Cystoadenocarcinoma',
                'Ductal carcinoma',
                'Large cell carcinoma',
                'Papillary infiltrating ductal carcinoma',
                'Renal spindle cell carcinoma',
                'Renal cell carcinoma',
                'Small cell bronchioalveolar carcinoma',
                'Squamous cell carcinoma',
                'Hypernephroma',
                'Infiltrating ductal carcinoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_2)))

Il y a 55 échantillons dans cette catégorie


Leucémie : 

In [9]:
category_3 = [
                'Acute lymphoblastic leukemia',
                'Chronic myelogenous leukemia',
                'Lymphoma',
                'Myeloma',
                'Pro myelocytic leukemia'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_3)))

Il y a 18 échantillons dans cette catégorie


Mélanome :

In [10]:
category_4 = [
                'Malignant amelanotic melanoma',
                'Malignant melanotic melanoma',
                'Melanotic melanoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_4)))

Il y a 23 échantillons dans cette catégorie


Cerveau :

In [11]:
category_5 = [
                'Astrocytoma',
                'Glial cell neoplasm',
                'Glioblastoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_5)))

Il y a 18 échantillons dans cette catégorie


**Vérifier si les écart-type et moyennes de ces catégories sont logiques, pour confirmer si on a bien choisi**

**Ajout de la catégorie**

In [12]:
dataDF_raw = pd.DataFrame(data=data[1:,1:], index=data[1:,0],columns=data[0,1:])
dataDF_raw.head()

,IDENTIFIER,GSM803615,GSM803674,GSM803733,GSM803616,GSM803675,GSM803734,GSM803617,GSM803676,GSM803735,...,GSM803785,GSM803671,GSM803729,GSM803786,GSM803672,GSM803730,GSM803787,GSM803673,GSM803731,GSM803788
1007_s_at,DDR1,7.98588,6.60953,7.28692,6.81414,7.17116,7.22437,8.30001,7.97986,6.8234,...,8.76158,8.87174,9.05203,9.02302,7.90743,7.85594,6.90531,10.799,10.8054,10.9599
1053_at,RFC2,8.91259,8.95833,8.78347,8.68783,8.47835,8.87411,8.48144,8.44854,8.00108,...,9.01109,7.99229,7.89066,7.78652,7.38044,7.94416,8.20352,7.82165,7.71562,7.79544
117_at,HSPA6,5.95818,5.9552,4.75223,4.96283,5.01497,4.78948,4.66964,4.70403,4.69634,...,5.32821,5.22383,5.24615,4.89699,5.5685,4.93727,5.18978,5.01333,4.98273,4.97768
121_at,PAX8,7.25647,7.34984,7.03978,7.23067,7.63878,7.45663,7.60143,7.56602,7.27995,...,7.44675,7.36011,7.30075,7.23286,7.3711,7.40892,7.23629,7.2943,7.2762,7.45836
1255_g_at,GUCA1A,2.70294,2.72227,2.66169,2.79379,2.97056,2.62163,3.09889,2.87228,2.76879,...,2.91059,2.67582,2.74977,2.68969,2.85014,2.794,2.74596,2.74609,2.75365,2.57236


In [13]:
# transposée pour avoir cellule avec les features de ML en lignes
dataDF = pd.DataFrame(data=data[1:,1:], index=data[1:,0],columns=data[0,1:]).transpose() 

dataDF = dataDF.drop('IDENTIFIER') # IDENTIFIER pas utilisé
dataDF = dataDF.iloc[:,:].astype('float64')
dataDF.insert(0, "category", 1,allow_duplicates=True) # création d'une colonne pour les catégorie de regroupement

In [14]:
# set the rigth category for each cell
all_categories = [category_1, category_2, category_3, category_4, category_5] # liste de listes
category_column = []

for index, row in dataDF.iterrows():
    for cat_index in range(len(all_categories)):
        if id_cond[index] in all_categories[cat_index]:
            category_column.append(cat_index + 1) # indices 1-5
            break      

dataDF['category'] = category_column
dataDF.head()

,category,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
GSM803615,3,7.98588,8.91259,5.95818,7.25647,2.70294,7.31083,4.93041,4.50100,2.79874,...,2.45076,3.53323,2.74954,3.15915,3.62218,3.36355,2.67324,2.72860,3.29524,2.80815
GSM803674,3,6.60953,8.95833,5.95520,7.34984,2.72227,6.82958,5.11308,4.64615,2.72531,...,2.45229,3.55865,2.97269,2.99266,3.77177,3.38203,2.88409,2.70408,3.29139,3.09778
GSM803733,3,7.28692,8.78347,4.75223,7.03978,2.66169,6.80481,5.10009,4.29090,2.78749,...,2.58546,3.25001,2.71918,2.97779,3.76105,3.27553,2.64535,2.74768,3.21011,2.99015
GSM803616,3,6.81414,8.68783,4.96283,7.23067,2.79379,6.90361,4.89782,4.54651,7.99732,...,2.48428,3.59843,2.90260,3.13846,3.91334,3.14986,3.16415,2.78068,3.22221,2.99241
GSM803675,3,7.17116,8.47835,5.01497,7.63878,2.97056,6.70489,5.29439,4.84493,7.48446,...,2.90313,3.56928,3.01864,2.95408,3.67145,3.89064,3.00442,2.72479,3.40582,3.01408


In [15]:
dataDF['1007_s_at'].describe()

count    174.000000
mean       9.198371
std        1.013861
min        6.598980
25%        8.653343
50%        9.248630
75%        9.887240
max       11.703400
Name: 1007_s_at, dtype: float64

**Vérification**

In [16]:
# séparation des cellules par catégories
cat1_df = dataDF[dataDF['category'] == 1]
cat2_df = dataDF[dataDF['category'] == 2] 
cat3_df = dataDF[dataDF['category'] == 3] 
cat4_df = dataDF[dataDF['category'] == 4] 
cat5_df = dataDF[dataDF['category'] == 5]

In [17]:
print("Nombre d'échantillons par catégorie")
print('1 :',len(cat1_df), '\n2 :',len(cat2_df), '\n3 :',len(cat3_df), '\n4 :',len(cat4_df), '\n5 :',len(cat5_df), '\n\n')    

Nombre d'échantillons par catégorie
1 : 60 
2 : 55 
3 : 18 
4 : 23 
5 : 18 




Ensuite, il faut séparer les données de test et d'entraînement :

![coucou](./images/test_train_diagram.png)

Ce diagramme se trouve ici si on veut le modifier : [draw.io](https://drive.google.com/file/d/1mcjHukPsXNBmaIeI8rkmtCZdjN7z8OEm/view?usp=sharing)

Pour cette étape, on peut s'inspirer du code du labo 3-4 de BBC : 

In [19]:
# Labo 3-4 BBC :
# divide the set into two sets: a train set and a test set for future validations
# idx_test = [5,11,19,28,54,61]  # Soit on prend un pourcentage soit on en choisit genre 3 par catégories
# idx_train = list(range(0,len(id_cond)+2)) 
# [idx_train.remove(i) for i in idx_test] # on enlève les données de test

# data_test = data[:,idx_test]
# data = data[:,idx_train]

# print(np.shape(data))
# print(np.shape(data_test))

Et Maintenant il faut voir pour ces écart-type

un endroit par là, il y a les fonctions à Guidoux qui diminue le nombre de variable de 50'000 a plus petit, pourquoi on utilise pas les méthodes du labo 3-4 ? Parce que c'était un problème à deux classes, et là on est multi classe ?


Les guidoux méthodes sont dans ce [google colab notebook](https://colab.research.google.com/drive/1rWg2k7-RtgJfzjacYNNY33m00WGDRxRq) à 
+ Méthode 1 - Unvariant Selection
+ Methode 4 - Recursive features elimination

Mais au premier abord, il faudrait échanger les lignes et les colonnes, et mettre les catégories dans un autre tableau

Même s'il manque les méthodes à Guidoux, ce serait bien de mettre les méthodes à Joël pour la classification, au moins une des 3 : 

+ Decision tree classifier
+ Support vector machine classifier
+ k-nearest neighbours classifier
